In [18]:
#import libraries
import os
import cv2
import numpy as np
import keras
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from keras.applications import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam 


Helper functions and variables

In [19]:
def list_labels(file):
    labels_file = open(file, "r")
    labels = []
    
    for line in labels_file:
        label = line.strip()
        labels.append(label)
    
    labels_file.close()
    
    return labels


#list of all labels
class_names = list_labels("./CamSDD/Labels.txt")
class_name_labels = {class_name:i for i, class_name in enumerate(class_names)}



def load_data(folder):
    Category = ["training", "test", "validation"]
    output = []
    
    for category in Category:
        print("Loading {}".format(category))
        path = os.path.join(folder, category)
        print(path)
        images = []
        labels = []
        
        for sub_folder in os.listdir(path):
            label = class_name_labels[sub_folder]
            
            #Iterating through all images
            for file in os.listdir(os.path.join(path, sub_folder)):
                
                #getting the image path
                img_path = os.path.join(os.path.join(path, sub_folder), file)
                
                #appending image and corresponding label
                images.append(cv2.imread(img_path))
                labels.append(label)
            
        #check that data type doesn't affect accuracy
        images = np.array(images, dtype='float32')
        labels = np.array(labels, dtype='int32')
        
        output.append((images, labels))
        
    return output



#displays 25 images with labels
def display_examples(class_names, images, labels):
    figsize = (20, 20)
    fig = plt.figure(figsize=figsize)
    fig.suptitle("Example of images", fontsize=16)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        # image = cv2.resize(images[i], figsize)
        plt.imshow(images[i].astype(np.uint8))
        plt.xlabel(class_names[labels[i]])
    plt.show()

Loading data

In [20]:
(train_images, train_labels), (test_images, test_labels), (validation_images, validation_labels)= load_data(".\\CamSDD")

Loading training
.\CamSDD\training
Loading test
.\CamSDD\test
Loading validation
.\CamSDD\validation


In [21]:
#shuffling train data
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

MemoryError: Unable to allocate 24.5 GiB for an array with shape (9898, 384, 576, 3) and data type float32

Visualize data

In [ ]:
display_examples(class_names, train_images, train_labels)

Creating model

In [15]:
#imports the mobilenet model and discards the last 1000 neuron layer.
#check how many neurons should be discarded
base_model=MobileNet(weights='imagenet',include_top=False) 

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024, activation='sigmoid')(x)
x=Dropout(0.7)(x)
output = Dense(30, activation="softmax")(x)

model=Model(inputs=base_model.input,outputs=output)

In [16]:
#checking model architecture
# print(model.summary())
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_2
1 conv1
2 conv1_bn
3 conv1_relu
4 conv_dw_1
5 conv_dw_1_bn
6 conv_dw_1_relu
7 conv_pw_1
8 conv_pw_1_bn
9 conv_pw_1_relu
10 conv_pad_2
11 conv_dw_2
12 conv_dw_2_bn
13 conv_dw_2_relu
14 conv_pw_2
15 conv_pw_2_bn
16 conv_pw_2_relu
17 conv_dw_3
18 conv_dw_3_bn
19 conv_dw_3_relu
20 conv_pw_3
21 conv_pw_3_bn
22 conv_pw_3_relu
23 conv_pad_4
24 conv_dw_4
25 conv_dw_4_bn
26 conv_dw_4_relu
27 conv_pw_4
28 conv_pw_4_bn
29 conv_pw_4_relu
30 conv_dw_5
31 conv_dw_5_bn
32 conv_dw_5_relu
33 conv_pw_5
34 conv_pw_5_bn
35 conv_pw_5_relu
36 conv_pad_6
37 conv_dw_6
38 conv_dw_6_bn
39 conv_dw_6_relu
40 conv_pw_6
41 conv_pw_6_bn
42 conv_pw_6_relu
43 conv_dw_7
44 conv_dw_7_bn
45 conv_dw_7_relu
46 conv_pw_7
47 conv_pw_7_bn
48 conv_pw_7_relu
49 conv_dw_8
50 conv_dw_8_bn
51 conv_dw_8_relu
52 conv_pw_8
53 conv_pw_8_bn
54 conv_pw_8_relu
55 conv_dw_9
56 conv_dw_9_bn
57 conv_dw_9_relu
58 conv_pw_9
59 conv_pw_9_bn
60 conv_pw_9_relu
61 conv_dw_10
62 conv_dw_10_bn
63 conv_dw_10_relu
64 conv_pw_10
65 conv_pw_1

In [17]:
# Freeze all layers except the ones we created above (check which method)
# for layer in model.layers[:-4]:
#     layer.trainable=False
for layer in base_model.layers:
    layer.trainable = False

In [22]:
#conclude: one of the two decay methods are incorrect
# check what loss function they used
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1485,
    decay_rate=0.1)

class PRINTLR(keras.callbacks.Callback):
    def __init__ (self, model):
        self.model=model
    def on_epoch_end(self, epoch, logs=None):
        lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
        print('\non epoch ',epoch + 1, ' lr was adjusted to ', lr)



model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='sparse_categorical_crossentropy')
model.fit(train_images, train_labels, batch_size=20, epochs=15, callbacks=[PRINTLR(model)])

# class ADJUSTLR(keras.callbacks.Callback):
#     def __init__ (self, model, freq, factor, verbose):
#         self.model=model
#         self.freq=freq
#         self.factor =factor
#         self.verbose=verbose
#         self.adj_epoch=freq
#     def on_epoch_end(self, epoch, logs=None):
#         if epoch + 1 == self.adj_epoch: # adjust the learning rate
#             lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
#             new_lr=lr * self.factor
#             self.adj_epoch +=self.freq
#             if self.verbose == 1:
#                 print('\non epoch ',epoch + 1, ' lr was adjusted from ', lr, ' to ', new_lr)
#             tf.keras.backend.set_value(self.model.optimizer.lr, new_lr) # set the learning rate in the optimizer
# freq=3
# factor=0.1
# verbose=1
# callbacks=[ADJUSTLR(model, freq, factor, verbose)]
# model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy')
# model.fit(train_images, train_labels, batch_size=20, epochs=15, callbacks=callbacks)

Epoch 1/15
495/495 [==============================] - 318s 639ms/step - loss: 2.3580
Epoch 2/15
495/495 [==============================] - 306s 619ms/step - loss: 2.0688
Epoch 3/15
495/495 [==============================] - ETA: 0s - loss: 1.8990
on epoch  3  lr was adjusted from  9.999999747378752e-05  to  9.999999747378752e-06
495/495 [==============================] - 298s 601ms/step - loss: 1.8990
Epoch 4/15
495/495 [==============================] - 299s 603ms/step - loss: 1.8155
Epoch 5/15
495/495 [==============================] - 300s 606ms/step - loss: 1.7939
Epoch 6/15
495/495 [==============================] - ETA: 0s - loss: 1.7818
on epoch  6  lr was adjusted from  9.999999747378752e-06  to  9.999999747378752e-07
495/495 [==============================] - 298s 603ms/step - loss: 1.7818
Epoch 7/15
495/495 [==============================] - 298s 601ms/step - loss: 1.7620
Epoch 8/15
495/495 [==============================] - 298s 602ms/step - loss: 1.7678
Epoch 9/15
495/495 [

Evaluating trained model

In [23]:
test_loss = model.evaluate(test_images, test_labels)

19/19 [==============================] - 21s 933ms/step - loss: 1.6270


In [24]:
predictions = model.predict(test_images)
pred_labels = np.argmax(predictions, axis=1)
print(classification_report(test_labels, pred_labels))

19/19 [==============================] - 18s 938ms/step
              precision    recall  f1-score   support

           0       0.62      0.25      0.36        20
           1       0.47      0.75      0.58        20
           2       0.33      0.05      0.09        20
           3       0.23      0.50      0.32        20
           4       0.33      0.05      0.09        20
           5       0.33      0.35      0.34        20
           6       0.52      0.75      0.61        20
           7       0.58      0.55      0.56        20
           8       0.66      0.95      0.78        20
           9       0.47      0.75      0.58        20
          10       0.78      0.70      0.74        20
          11       0.71      0.50      0.59        20
          12       0.70      0.80      0.74        20
          13       0.63      0.85      0.72        20
          14       0.60      0.75      0.67        20
          15       0.63      0.60      0.62        20
          16       0.57  